In [4]:
from core.models import r, Game, RelatedGame
from core.config import get_settings
from redis.commands.search.query import Query, NumericFilter
from redis.commands.search import aggregation
from redis.commands.search import reducers
from datetime import datetime, timezone
from scrapings.base import ScrapingBase
from scrapings import tonybet
from scrapings import dafabet
import websocket

In [11]:
settings = get_settings()
url = 'https://www.dafabet.com/es/dfgoal/sports/240-football/23234-international/212108-world-cup/37234773-daily-odds-booster/event-36195030-croatia-vs-morocco'
uri = f'{settings.WS}/games/dafabet.com?game_id={url}'

In [16]:
ws_bet = websocket.WebSocket()
ws_bet.connect(uri)